In [1]:
import os
import pandas as pd
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .config("spark.driver.memory", "2g") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/07 08:01:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Set the dataset and file paths
csv_file = 'data/us-accidents.csv'

dataset_name = 'sobhanmoosavi/us-accidents'
os.system(f'kaggle datasets download -d {dataset_name} -p data') 

 78%|███████▊  | 209M/269M [00:00<00:00, 292MB/s] 


 88%|████████▊ | 238M/269M [00:00<00:00, 291MB/s]
 99%|█████████▉| 267M/269M [00:00<00:00, 295MB/s]
100%|██████████| 269M/269M [00:00<00:00, 288MB/s]


0

In [4]:
!zcat data/us-accidents.zip > data/us-accidents.csv


gzip: data/us-accidents.zip: invalid compressed data--length error


In [5]:
df_accidents_pd = pd.read_csv(csv_file, nrows=100)

In [6]:
df_accidents_pd['Weather_Condition'] = df_accidents_pd['Weather_Condition'].astype('str')
df_accidents_pd = df_accidents_pd.rename(columns={
        'Distance(mi)': 'Distance_miles',
        'Temperature(F)': 'Temperature_F',
        'Wind_Chill(F)': 'Wind_Chill_F',
        'Humidity(%)': 'Humidity_perc',
        'Pressure(in)': 'Pressure_inches',
        'Visibility(mi)': 'Visibility_miles',
        'Wind_Speed(mph)': 'Wind_Speed_mph',
        'Precipitation(in)': 'Precipitation_inches'
    })

In [7]:
spark.createDataFrame(df_accidents_pd).schema

StructType([StructField('ID', StringType(), True), StructField('Severity', LongType(), True), StructField('Start_Time', StringType(), True), StructField('End_Time', StringType(), True), StructField('Start_Lat', DoubleType(), True), StructField('Start_Lng', DoubleType(), True), StructField('End_Lat', DoubleType(), True), StructField('End_Lng', DoubleType(), True), StructField('Distance_miles', DoubleType(), True), StructField('Description', StringType(), True), StructField('Number', DoubleType(), True), StructField('Street', StringType(), True), StructField('Side', StringType(), True), StructField('City', StringType(), True), StructField('County', StringType(), True), StructField('State', StringType(), True), StructField('Zipcode', StringType(), True), StructField('Country', StringType(), True), StructField('Timezone', StringType(), True), StructField('Airport_Code', StringType(), True), StructField('Weather_Timestamp', StringType(), True), StructField('Temperature_F', DoubleType(), Tru

In [8]:
from pyspark.sql import types

In [9]:
accidents_schema = types.StructType([
    types.StructField('ID', types.StringType(), True),
    types.StructField('Severity', types.IntegerType(), True),
    types.StructField('Start_Time', types.TimestampType(), True),
    types.StructField('End_Time', types.TimestampType(), True),
    types.StructField('Start_Lat', types.DoubleType(), True),
    types.StructField('Start_Lng', types.DoubleType(), True),
    types.StructField('End_Lat', types.DoubleType(), True),
    types.StructField('End_Lng', types.DoubleType(), True),
    types.StructField('Distance_miles', types.DoubleType(), True),
    types.StructField('Description', types.StringType(), True),
    types.StructField('Number', types.DoubleType(), True),
    types.StructField('Street', types.StringType(), True),
    types.StructField('Side', types.StringType(), True),
    types.StructField('City', types.StringType(), True),
    types.StructField('County', types.StringType(), True),
    types.StructField('State', types.StringType(), True),
    types.StructField('Zipcode', types.StringType(), True),
    types.StructField('Country', types.StringType(), True),
    types.StructField('Timezone', types.StringType(), True),
    types.StructField('Airport_Code', types.StringType(), True),
    types.StructField('Weather_Timestamp', types.TimestampType(), True),
    types.StructField('Temperature_F', types.DoubleType(), True),
    types.StructField('Wind_Chill_F', types.DoubleType(), True),
    types.StructField('Humidity_perc', types.DoubleType(), True),
    types.StructField('Pressure_inches', types.DoubleType(), True),
    types.StructField('Visibility_miles', types.DoubleType(), True),
    types.StructField('Wind_Direction', types.StringType(), True),
    types.StructField('Wind_Speed_mph', types.DoubleType(), True),
    types.StructField('Precipitation_inches', types.DoubleType(), True),
    types.StructField('Weather_Condition', types.StringType(), True),
    types.StructField('Amenity', types.BooleanType(), True),
    types.StructField('Bump', types.BooleanType(), True),
    types.StructField('Crossing', types.BooleanType(), True),
    types.StructField('Give_Way', types.BooleanType(), True),
    types.StructField('Junction', types.BooleanType(), True),
    types.StructField('No_Exit', types.BooleanType(), True),
    types.StructField('Railway', types.BooleanType(), True),
    types.StructField('Roundabout', types.BooleanType(), True),
    types.StructField('Station', types.BooleanType(), True),
    types.StructField('Stop', types.BooleanType(), True),
    types.StructField('Traffic_Calming', types.BooleanType(), True),
    types.StructField('Traffic_Signal', types.BooleanType(), True),
    types.StructField('Turning_Loop', types.BooleanType(), True),
    types.StructField('Sunrise_Sunset', types.StringType(), True),
    types.StructField('Civil_Twilight', types.StringType(), True),
    types.StructField('Nautical_Twilight', types.StringType(), True),
    types.StructField('Astronomical_Twilight', types.StringType(), True)
])

In [10]:
input_path = "data/us-accidents.csv"
output_path = "data/pq/"

df_accidents = spark.read \
    .option("header","true") \
    .schema(accidents_schema) \
    .csv(input_path) \

df_accidents \
    .repartition(24) \
    .write.parquet(output_path, mode='overwrite')

23/03/07 08:02:37 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


23/03/07 08:02:39 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: ID, Severity, Start_Time, End_Time, Start_Lat, Start_Lng, End_Lat, End_Lng, Distance(mi), Description, Number, Street, Side, City, County, State, Zipcode, Country, Timezone, Airport_Code, Weather_Timestamp, Temperature(F), Wind_Chill(F), Humidity(%), Pressure(in), Visibility(mi), Wind_Direction, Wind_Speed(mph), Precipitation(in), Weather_Condition, Amenity, Bump, Crossing, Give_Way, Junction, No_Exit, Railway, Roundabout, Station, Stop, Traffic_Calming, Traffic_Signal, Turning_Loop, Sunrise_Sunset, Civil_Twilight, Nautical_Twilight, Astronomical_Twilight
 Schema: ID, Severity, Start_Time, End_Time, Start_Lat, Start_Lng, End_Lat, End_Lng, Distance_miles, Description, Number, Street, Side, City, County, State, Zipcode, Country, Timezone, Airport_Code, Weather_Timestamp, Temperature_F, Wind_Chill_F, Humidity_perc, Pressure_inches, Visibility_miles, Wind_Direction, Wind_Speed_mph, Precipitation_in

In [12]:
df_accidents.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Severity: integer (nullable = true)
 |-- Start_Time: timestamp (nullable = true)
 |-- End_Time: timestamp (nullable = true)
 |-- Start_Lat: double (nullable = true)
 |-- Start_Lng: double (nullable = true)
 |-- End_Lat: double (nullable = true)
 |-- End_Lng: double (nullable = true)
 |-- Distance_miles: double (nullable = true)
 |-- Description: string (nullable = true)
 |-- Number: double (nullable = true)
 |-- Street: string (nullable = true)
 |-- Side: string (nullable = true)
 |-- City: string (nullable = true)
 |-- County: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Zipcode: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Timezone: string (nullable = true)
 |-- Airport_Code: string (nullable = true)
 |-- Weather_Timestamp: timestamp (nullable = true)
 |-- Temperature_F: double (nullable = true)
 |-- Wind_Chill_F: double (nullable = true)
 |-- Humidity_perc: double (nullable = true)
 |-- Pressur